In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from typing import Tuple
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import cohen_kappa_score


### MLP class for Descriptor features

In [10]:
class DescriptorMLP(nn.Module):
    def __init__(self, input_size, hidden_nodes, hidden_layers, dropout_rate):
        super(DescriptorMLP, self).__init__()
        self.hidden_layers = hidden_layers
        self.hidden_nodes = hidden_nodes
        self.dropout_rate = dropout_rate
        
        # Layers
        self.fc1 = nn.Linear(input_size, hidden_nodes)
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.fc_layers = nn.ModuleList([
            nn.Linear(hidden_nodes, hidden_nodes) for _ in range(hidden_layers - 1)
        ])
        self.fc_out = nn.Linear(hidden_nodes, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)

        for layer in self.fc_layers:
            x = torch.relu(layer(x))
            x = self.dropout(x)
        
        x = torch.sigmoid(self.fc_out(x))
        return x

### Class DescriptorDataset for convertion in torch.tensor

In [11]:
class DescriptorDataset(Dataset):
    def __init__(self, X_data: np.ndarray, y_data: np.ndarray):
        self.X = torch.tensor(X_data, dtype=torch.float32)
        self.y = torch.tensor(y_data, dtype=torch.float32).view(-1, 1)
        
    def __len__(self) -> int:
        return len(self.X)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.X[idx], self.y[idx]

### Prediction fct°

In [12]:
def predict_mlp(model: nn.Module, X_test: np.ndarray) -> np.ndarray:
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_pred = model(X_test_tensor).squeeze()
        return y_pred.round().numpy()


In [13]:
def train_dnn_with_dataloader(X_train, y_train, model, batch_size=32, learning_rate=0.001, epochs=100, save_path="descriptor_based_dnn.pth"):
    # loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoader
    dataset = DescriptorDataset(X_train, y_train)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Training loop with tqdm progress bar
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        epoch_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=True)
        
        for inputs, labels in epoch_bar:
            # Forward pass
            y_pred = model(inputs)
            
            # Compute loss
            loss = criterion(y_pred, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")
            train_acc = np.mean(predict_mlp(model, X_train) == y_train)
            print(f"Train Accuracy: {train_acc:.4f}")
    
    # Save the model
    torch.save(model.state_dict(), save_path)
    print(f"Model trained and saved at {save_path}")

    return model

In [14]:
def run_descriptor_based_DNN(smiles_list,C_total, trained_model):
    # Convert data to PyTorch tensors
    C_total_tensor = torch.tensor(C_total, dtype=torch.float32)

    # Model initialization
    input_size = C_total.shape[1]  # number of features
    hidden_nodes = 892
    hidden_layers = 4
    dropout_rate = 0.2
    
    model = DescriptorMLP(input_size, hidden_nodes, hidden_layers, dropout_rate)
    
    # Load the trained model (assuming it is saved in .pt format)
    model.load_state_dict(torch.load(trained_model))
    model.eval()

    results = {}
    
    with torch.no_grad():
        # Make predictions
        y_pred = model(C_total_tensor).squeeze()

        for i, smi in enumerate(smiles_list):
            prob = y_pred[i].item()  # Convert tensor to native Python float
            results[smi] = prob

    return results

### Descriptors to extract

In [15]:
def extract_selected_features(data):
    # Liste des descripteurs à extraire
    descriptors_to_extract = [
        'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v',
        'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'EState_VSA1', 'EState_VSA10',
        'EState_VSA11', 'EState_VSA2', 'EState_VSA3', 'EState_VSA4', 'EState_VSA5', 'EState_VSA6',
        'EState_VSA7', 'EState_VSA8', 'EState_VSA9', 'ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan2',
        'FpDensityMorgan3', 'FractionCSP3', 'HallKierAlpha', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Kappa1',
        'Kappa2', 'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MaxEStateIndex',
        'MaxPartialCharge', 'MinAbsEStateIndex', 'MinAbsPartialCharge', 'MinEStateIndex', 'MinPartialCharge',
        'MolLogP', 'MolMR', 'MolWt', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles',
        'NumAliphaticRings', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings',
        'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRadicalElectrons', 'NumRotatableBonds',
        'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 'NumSaturatedRings', 'NumValenceElectrons',
        'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2',
        'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9',
        'RingCount', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6',
        'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12',
        'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8',
        'SlogP_VSA9', 'TPSA', 'VSA_EState1', 'VSA_EState10', 'VSA_EState2', 'VSA_EState3', 'VSA_EState4',
        'VSA_EState5', 'VSA_EState6', 'VSA_EState7', 'VSA_EState8', 'VSA_EState9', 'fr_Al_COO', 'fr_Al_OH',
        'fr_Al_OH_noTert', 'fr_ArN', 'fr_Ar_COO', 'fr_Ar_N', 'fr_Ar_NH', 'fr_Ar_OH', 'fr_COO', 'fr_COO2',
        'fr_C_O', 'fr_C_O_noCOO', 'fr_C_S', 'fr_HOCCN', 'fr_Imine', 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_N_O',
        'fr_Ndealkylation1', 'fr_Ndealkylation2', 'fr_Nhpyrrole', 'fr_SH', 'fr_aldehyde', 'fr_alkyl_carbamate',
        'fr_alkyl_halide', 'fr_allylic_oxid', 'fr_amide', 'fr_amidine', 'fr_aniline', 'fr_aryl_methyl', 'fr_azide',
        'fr_azo', 'fr_barbitur', 'fr_benzene', 'fr_benzodiazepine', 'fr_bicyclic', 'fr_diazo', 'fr_dihydropyridine',
        'fr_epoxide', 'fr_ester', 'fr_ether', 'fr_furan', 'fr_guanido', 'fr_halogen', 'fr_hdrzine', 'fr_hdrzone',
        'fr_imidazole', 'fr_imide', 'fr_isocyan', 'fr_isothiocyan', 'fr_ketone', 'fr_ketone_Topliss', 'fr_lactam',
        'fr_lactone', 'fr_methoxy', 'fr_morpholine', 'fr_nitrile', 'fr_nitro', 'fr_nitro_arom', 'fr_nitro_arom_nonortho',
        'fr_nitroso', 'fr_oxazole', 'fr_oxime', 'fr_para_hydroxylation', 'fr_phenol', 'fr_phenol_noOrthoHbond',
        'fr_phos_acid', 'fr_phos_ester', 'fr_piperdine', 'fr_piperzine', 'fr_priamide', 'fr_prisulfonamd', 'fr_pyridine',
        'fr_quatN', 'fr_sulfide', 'fr_sulfonamd', 'fr_sulfone', 'fr_term_acetylene', 'fr_tetrazole', 'fr_thiazole',
        'fr_thiocyan', 'fr_thiophene', 'fr_unbrch_alkane', 'fr_urea', 'qed'
    ]
    C_total = data[descriptors_to_extract]
    smiles_list = data['smiles']
    return C_total, smiles_list


## Main

In [ ]:

# Load data
data = pd.read_csv("data/train.csv")
train80 = pd.read_csv("train_data_80.csv")
train2O = pd.read_csv("train_data_20.csv")
X_80, smiles_list_80 = extract_selected_features(train80)
X_20, smiles_list_20 = extract_selected_features(train2O)

# Scale the data
X_80 = StandardScaler().fit_transform(X_80)
X_20 = StandardScaler().fit_transform(X_20)

y_80 = train80['class']
y_20 = train2O['class']

# Split data
X_train = X_80
y_train = y_80
X_test = X_20
y_test = y_20

# Model configuration
config = {
    "input_size": X_train.shape[1],
    "hidden_nodes": 892,
    "hidden_layers": 4,
    "dropout_rate": 0.3,
    "learning_rate": 0.0001,
    "epochs": 100,
    "batch_size": 32,
    "save_path": "descriptor_based_dnn.pth"
}

# Initialize and train model
model = DescriptorMLP(
    config["input_size"],
    config["hidden_nodes"],
    config["hidden_layers"],
    config["dropout_rate"]
)
train_params = {key: config[key] for key in ["batch_size", "learning_rate", "epochs", "save_path"]}
model = train_dnn_with_dataloader(X_train, y_train, model, **train_params)
# Predict on test set
y_pred = predict_mlp(model, X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy on test set: {accuracy:.4f}")
print(f"Cohen's Kappa: {cohen_kappa_score(y_test, y_pred):.4f}")



Epoch 10/100: 100%|██████████| 59/59 [00:00<00:00, 79.09it/s]


Epoch 10/100, Loss: 0.3727
Train Accuracy: 0.8816


Epoch 20/100: 100%|██████████| 59/59 [00:00<00:00, 89.94it/s]


Epoch 20/100, Loss: 0.2427
Train Accuracy: 0.9655


Epoch 30/100: 100%|██████████| 59/59 [00:00<00:00, 88.42it/s]


...

Epoch 80/100, Loss: 0.0315
Train Accuracy: 0.9984


Epoch 90/100: 100%|██████████| 59/59 [00:00<00:00, 88.41it/s]


Epoch 90/100, Loss: 0.0303
Train Accuracy: 0.9968


Epoch 100/100: 100%|██████████| 59/59 [00:01<00:00, 37.14it/s]


Epoch 100/100, Loss: 0.0299
Train Accuracy: 0.9989
Model trained and saved at descriptor_based_dnn.pth
Accuracy on test set: 0.7554
Cohen's Kappa: 0.5108


In [9]:
# Ajout des imports nécessaires
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score
import numpy as np

# Fonction pour entraîner et évaluer le modèle avec cross-validation
def cross_validate_dnn(X, y, config, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    accuracies = []
    kappas = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\nFold {fold + 1}/{n_splits}")
        
        # Séparation des données
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Initialisation du modèle
        model = DescriptorMLP(
            config["input_size"],
            config["hidden_nodes"],
            config["hidden_layers"],
            config["dropout_rate"]
        )
        
        # Entraînement
        train_params = {
            "batch_size": config["batch_size"],
            "learning_rate": config["learning_rate"],
            "epochs": config["epochs"],
            "save_path": f"model_fold_{fold}.pth"
        }
        model = train_dnn_with_dataloader(X_train, y_train, model, **train_params)
        
        # Évaluation
        y_pred = predict_mlp(model, X_val)
        acc = accuracy_score(y_val, y_pred)
        kappa = cohen_kappa_score(y_val, y_pred)
        
        accuracies.append(acc)
        kappas.append(kappa)
        
        print(f"Fold {fold + 1} - Accuracy: {acc:.4f}, Kappa: {kappa:.4f}")
    
    # Affichage des résultats
    print("\nCross-validation results:")
    print(f"Mean Accuracy: {np.mean(accuracies):.4f} (±{np.std(accuracies):.4f})")
    print(f"Mean Kappa: {np.mean(kappas):.4f} (±{np.std(kappas):.4f})")
    
    return accuracies, kappas

# Configuration du modèle
config = {
    "input_size": X_80.shape[1],
    "hidden_nodes": 892,
    "hidden_layers": 4,
    "dropout_rate": 0.3,
    "learning_rate": 0.0001,
    "epochs": 100,
    "batch_size": 32
}

# Chargement de toutes les données
X_all = np.vstack([X_80, X_20])
y_all = np.concatenate([y_80, y_20])

# Exécution de la cross-validation
accuracies, kappas = cross_validate_dnn(X_all, y_all, config, n_splits=5)

# Entraînement final sur toutes les données si nécessaire
final_model = DescriptorMLP(
    config["input_size"],
    config["hidden_nodes"],
    config["hidden_layers"],
    config["dropout_rate"]
)
final_model = train_dnn_with_dataloader(X_all, y_all, final_model, 
                                      batch_size=config["batch_size"],
                                      learning_rate=config["learning_rate"],
                                      epochs=config["epochs"],
                                      save_path="final_model.pth")


Fold 1/5


Epoch 10/100: 100%|██████████| 236/236 [00:02<00:00, 89.06it/s]


Epoch 10/100, Loss: 0.3544
Train Accuracy: 0.8841


Epoch 20/100: 100%|██████████| 236/236 [00:02<00:00, 93.51it/s]


Epoch 20/100, Loss: 0.2443
Train Accuracy: 0.9422


...

Epoch 80/100, Loss: 0.0867
Train Accuracy: 0.9844


Epoch 90/100: 100%|██████████| 295/295 [00:03<00:00, 96.77it/s] 


Epoch 90/100, Loss: 0.0789
Train Accuracy: 0.9894


Epoch 100/100: 100%|██████████| 295/295 [00:03<00:00, 92.92it/s]


Epoch 100/100, Loss: 0.0734
Train Accuracy: 0.9895
Model trained and saved at final_model.pth


In [17]:
# Load data
data = pd.read_csv("data/train.csv")
train80 = pd.read_csv("train_data_80.csv")
train2O = pd.read_csv("train_data_20.csv")
X_80, smiles_list_80 = extract_selected_features(train80)
X_20, smiles_list_20 = extract_selected_features(train2O)

# Scale the data
X_80 = StandardScaler().fit_transform(X_80)
X_20 = StandardScaler().fit_transform(X_20)

y_80 = train80['class']
y_20 = train2O['class']

# Split data
X_train = X_80
y_train = y_80
X_test = X_20
y_test = y_20

# Model configuration
config = {
    "input_size": X_train.shape[1],
    "hidden_nodes": 892,
    "hidden_layers": 4,
    "dropout_rate": 0.3,
    "learning_rate": 0.0001,
    "epochs": 100,
    "batch_size": 32,
    "save_path": "descriptor_based_dnn.pth"
}

# Initialize and train model
model = DescriptorMLP(
    config["input_size"],
    config["hidden_nodes"],
    config["hidden_layers"],
    config["dropout_rate"]
)
train_params = {key: config[key] for key in ["batch_size", "learning_rate", "epochs", "save_path"]}
model = train_dnn_with_dataloader(X_train, y_train, model, **train_params)
# Predict on test set
y_pred = predict_mlp(model, X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy on test set: {accuracy:.4f}")
print(f"Cohen's Kappa: {cohen_kappa_score(y_test, y_pred):.4f}")



Epoch 9/100:  66%|██████▌   | 156/236 [00:02<00:01, 69.11it/s]


KeyboardInterrupt: 

In [23]:
# Calcul des probabilités prédites pour chaque molécule du test set
model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_proba = model(X_test_tensor).squeeze().numpy()

# Création du DataFrame avec smiles et proba
df_pred = pd.DataFrame({
    "smiles": smiles_list_20.values,
    "proba": y_proba
})

# Sauvegarde au format CSV
df_pred.to_csv("mlp_desc_proba.csv", index=False)

In [25]:

# Charger le fichier test
test_df = pd.read_csv("/Users/rayanedakhlaoui/Desktop/VivaAfricAI/Final/test/test_1.csv")
# Extraire les features du test
X_test_submit, smiles_test_submit = extract_selected_features(test_df)

# Appliquer le même scaler que pour le train (ici, on refait un fit_transform, mais idéalement il faudrait utiliser le scaler du train)
X_test_submit = StandardScaler().fit_transform(X_test_submit)

# Model configuration
config = {
    "input_size": X_train.shape[1],
    "hidden_nodes": 892,
    "hidden_layers": 4,
    "dropout_rate": 0.3,
    "learning_rate": 0.0001,
    "epochs": 100,
    "batch_size": 32,
    "save_path": "descriptor_based_dnn.pth"
}


# Charger le modèle entraîné
submission_model = DescriptorMLP(
    config["input_size"],
    config["hidden_nodes"],
    config["hidden_layers"],
    config["dropout_rate"]
)
submission_model.load_state_dict(torch.load(config["save_path"]))
submission_model.eval()

# Prédire les probabilités
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test_submit, dtype=torch.float32)
    y_pred_submit = submission_model(X_test_tensor).squeeze().numpy()

# Générer le fichier de soumission
submission_df = pd.DataFrame({
    "smiles": smiles_test_submit.values,
    "proba": y_pred_submit
})
submission_df.to_csv("prediction_mlp.csv", index=False)
